In [1]:
# coding: utf-8
from __future__ import print_function, division, absolute_import
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# !rm -fvr logdir processed

In [3]:
!mkdir -p logdir/train logdir/test

In [4]:
from tensorflow.examples.tutorials.mnist.input_data \
  import read_data_sets

In [5]:
mnist = read_data_sets('./mnist', one_hot=False)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz


In [6]:
INPUT_UNITS = 28
NUM_HIDDEN_UNITS = 31
BATCH_SIZE = 128
MAX_SEQ_LEN = 28

In [7]:
train_loop_count = mnist.train.num_examples // BATCH_SIZE
test_loop_count  = mnist.test.num_examples // BATCH_SIZE

In [8]:
from minimalrnn import MinimalRNNCell

In [9]:
class MnistRnn:
    """
    """
    def __init__(self, inputs, labels):
        """
        """
        cell = MinimalRNNCell(NUM_HIDDEN_UNITS)
        
        sequence_length = [MAX_SEQ_LEN] * BATCH_SIZE
        
        last, states = tf.nn.dynamic_rnn(
            cell,
            inputs,
            sequence_length=sequence_length,
            dtype=tf.float32)
        rnn_output = last[:,MAX_SEQ_LEN-1,:]
        outputs    = tf.layers.dense(rnn_output, 10)
        
        loss       = tf.losses.sparse_softmax_cross_entropy(
            labels,
            outputs)
        optimize   = tf.train.AdamOptimizer(learning_rate=0.001). \
            minimize(loss)
            
        preds      = tf.argmax(outputs, axis=1)
        errors     = tf.count_nonzero(labels - preds)
        accuracy   = 1.0 - tf.cast(errors,tf.float32) / \
                        tf.cast(tf.size(preds), tf.float32)
        
        self.inputs   = inputs
        self.labels   = labels
        self.outputs  = outputs
        self.loss     = loss
        self.optimize = optimize
        self.accuracy = accuracy

In [10]:
tf.reset_default_graph()
inputs_ = tf.placeholder(tf.float32,
                         [BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS],
                         name='inputs')
labels_ = tf.placeholder(tf.int64,
                         [BATCH_SIZE],
                         name='labels')

model   = MnistRnn(inputs_, labels_)

In [11]:
config = tf.ConfigProto(gpu_options={'allow_growth':True})
sess   = tf.InteractiveSession(config=config)

tf.global_variables_initializer().run()

In [12]:
def train(max_epochs, train_writer=None, test_writer=None):
    """
    """
    
    step = 0
    
    for ep in range(max_epochs):
        for i in range(train_loop_count):
            offs = i * BATCH_SIZE
            batch_input = mnist.train.images[offs:offs+BATCH_SIZE,:]. \
                            reshape([BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS])
            batch_label = mnist.train.labels[offs:offs+BATCH_SIZE]
            _, loss, accuracy = sess.run(
                [model.optimize, model.loss, model.accuracy],
                 feed_dict = {
                     model.inputs: batch_input,
                     model.labels: batch_label })
            step += 1
            if train_writer:
                summary = tf.Summary(
                    value = [
                        tf.Summary.Value(tag='accuracy',simple_value=accuracy),
                        tf.Summary.Value(tag='loss',simple_value=loss),
                    ])
                train_writer.add_summary(summary, global_step=step)
            if step % 250 == 0:
                print('[trn] ep {:d}, step {:d}, loss {:.5f}, accu {:.5f}'.format(
                    ep + 1, step, loss, accuracy))
                
        test_accuracies = []
        for i in range(test_loop_count):
            offs = i * BATCH_SIZE
            batch_input = mnist.train.images[offs:offs+BATCH_SIZE,:]. \
                            reshape([BATCH_SIZE, MAX_SEQ_LEN, INPUT_UNITS])
            batch_label = mnist.train.labels[offs:offs+BATCH_SIZE]
            accuracy, = sess.run([model.accuracy],
                                 feed_dict = {
                                     model.inputs: batch_input,
                                     model.labels: batch_label})
            test_accuracies.append(accuracy)
            if test_writer:
                summary = tf.Summary(
                    value = [
                        tf.Summary.Value(tag='accuracy',simple_value=accuracy),
                    ])
                test_writer.add_summary(summary, global_step=step)
            if i % 250 == 0:
                print('[tst] ep {:d}, step {:d}, accu {:.5f}'.format(
                    ep + 1, step, np.mean(test_accuracies)))


In [13]:
train_writer = tf.summary.FileWriter('logdir/train', 
                                     graph=tf.get_default_graph())
test_writer  = tf.summary.FileWriter('logdir/test',
                                     graph=tf.get_default_graph())

In [14]:
# tf.get_default_graph().finalize()
train(10, train_writer, test_writer)

[trn] ep 1, step 250, loss 1.58229, accu 0.51562
[tst] ep 1, step 429, accu 0.75000
[trn] ep 2, step 500, loss 0.91454, accu 0.72656
[trn] ep 2, step 750, loss 0.68663, accu 0.77344
[tst] ep 2, step 858, accu 0.85156
[trn] ep 3, step 1000, loss 0.49808, accu 0.89844
[trn] ep 3, step 1250, loss 0.25797, accu 0.92969
[tst] ep 3, step 1287, accu 0.89062
[trn] ep 4, step 1500, loss 0.30145, accu 0.89844
[tst] ep 4, step 1716, accu 0.90625
[trn] ep 5, step 1750, loss 0.25783, accu 0.92969
[trn] ep 5, step 2000, loss 0.35933, accu 0.92188
[tst] ep 5, step 2145, accu 0.91406
[trn] ep 6, step 2250, loss 0.26601, accu 0.91406
[trn] ep 6, step 2500, loss 0.46013, accu 0.85938
[tst] ep 6, step 2574, accu 0.91406
[trn] ep 7, step 2750, loss 0.24249, accu 0.93750
[trn] ep 7, step 3000, loss 0.23225, accu 0.90625
[tst] ep 7, step 3003, accu 0.92969
[trn] ep 8, step 3250, loss 0.17944, accu 0.93750
[tst] ep 8, step 3432, accu 0.93750
[trn] ep 9, step 3500, loss 0.17169, accu 0.96094
[trn] ep 9, step 

In [15]:
# !tensorboard --logdir logdir

In [16]:
!mkdir -p processed

In [17]:
%%file processed/label.txt
[0:Zero]
[1:One]
[2:Two]
[3:Three]
[4:Four]
[5:Five]
[6:Six]
[7:Seven]
[8:Eight]
[9:Nine]

Overwriting processed/label.txt


In [18]:
t_vars = {v.name:v for v in tf.trainable_variables()}
embedding_var = tf.Variable(
    t_vars[u'dense/kernel:0'].eval().transpose(),
    name='embedding')
embedding_var.initializer.run()

In [19]:
from tensorflow.contrib.tensorboard.plugins import projector

In [20]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
embedding.metadata_path = './label.txt'

summary_writer = tf.summary.FileWriter('./processed')
projector.visualize_embeddings(summary_writer, config)

saver = tf.train.Saver([embedding_var])
saver.save(sess, './processed/save')

'./processed/save'

In [21]:
# !tensorboard --logdir processed